In [8]:
# Code based on: https://github.com/Lightning-AI/lightning

import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import lightning as L
import torch.utils.data as data_utils

In [9]:
class LitAutoEncoder(L.LightningModule):

    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 64),
            nn.ReLU(),
            nn.Linear(64, 3)
        )
        self.decoder = nn.Sequential(
            nn.Linear(3, 64),
            nn.ReLU(),
            nn.Linear(64, 28*28)
        )

    def forward(self, x):
        # In lightning, forward defines the prediction/inference actions
        embedding = self.encoder(x)
        return embedding

    def training_step(self, batch, batch_idx):
        # Training_step defined the train loop.
        # It is independent of forward().
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        # Just a simple encoding and decoding of the data and therefore no labels are needed (autoencoder).
        loss = F.mse_loss(x_hat, x)
        # Logging to TensorBoard by default.
        self.log('train_loss', loss)
        return loss
    
    def test_step(self, batch, batch_idx):
        # Same definition as training_step.
        # It is independent of forward().
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        # Just a simple encoding and decoding of the data and therefore no labels are needed (autoencoder).
        loss = F.mse_loss(x_hat, x)
        # Logging to TensorBoard by default.
        self.log('test_loss', loss)
        return loss
    
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [10]:
dataset = MNIST(os.getcwd(), download=True, transform=transforms.ToTensor())
indices_tr = torch.arange(1000)
indices_tt = torch.arange(1000,1200)
tr = data_utils.Subset(dataset, indices_tr)
tt = data_utils.Subset(dataset, indices_tt)
train_loader = DataLoader(tr, num_workers=4)
test_loader = DataLoader(tt, num_workers=4)

100%|██████████| 9912422/9912422 [00:00<00:00, 12393656.72it/s]


Extracting c:\Users\m-gre\OneDrive\01 - PROJECTS\Advanced Machine Learning\AML_Tutorial\MNIST\raw\train-images-idx3-ubyte.gz to c:\Users\m-gre\OneDrive\01 - PROJECTS\Advanced Machine Learning\AML_Tutorial\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 29437592.67it/s]


Extracting c:\Users\m-gre\OneDrive\01 - PROJECTS\Advanced Machine Learning\AML_Tutorial\MNIST\raw\train-labels-idx1-ubyte.gz to c:\Users\m-gre\OneDrive\01 - PROJECTS\Advanced Machine Learning\AML_Tutorial\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 3951829.55it/s] 


Extracting c:\Users\m-gre\OneDrive\01 - PROJECTS\Advanced Machine Learning\AML_Tutorial\MNIST\raw\t10k-images-idx3-ubyte.gz to c:\Users\m-gre\OneDrive\01 - PROJECTS\Advanced Machine Learning\AML_Tutorial\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting c:\Users\m-gre\OneDrive\01 - PROJECTS\Advanced Machine Learning\AML_Tutorial\MNIST\raw\t10k-labels-idx1-ubyte.gz to c:\Users\m-gre\OneDrive\01 - PROJECTS\Advanced Machine Learning\AML_Tutorial\MNIST\raw



In [11]:
# Init model
autoencoder = LitAutoEncoder()

# Most basic trainer which uses good defaults (auto-tensorboard, checkpoints, logs, and more)
trainer = L.Trainer(max_epochs=10)
trainer.fit(autoencoder, train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: c:\Users\m-gre\OneDrive\01 - PROJECTS\Advanced Machine Learning\AML_Tutorial\lightning_logs

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 50.4 K
1 | decoder | Sequential | 51.2 K
---------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [12]:
# Test model
trainer.test(autoencoder, test_loader)

Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.04543587192893028    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.04543587192893028}]